In [1]:
import tensorflow as tf 
import keras as k
import tensorflow_datasets as tfds
import numpy as np

d:\system_program\python3.11\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def sample_model():
    x = k.layers.Input(shape=(784,))
    l1 = k.layers.Dense(128, activation='relu')(x)
    l2 = k.layers.Dense(128, activation='relu')(l1)
    out = k.layers.Dense(10, activation='softmax')(l2)
    model = k.models.Model(inputs=x, outputs=out)
    return model

model = sample_model()


In [3]:
# give a try for keras model saveing and reload
# keras3 使用 .keras作為附檔名，且儲存了模型的yaml。這波要測試是否可以不用透過宣告模型計算圖，
# 直接讀取.keras就能運作

model.save("sample.keras")
reload_model = k.models.load_model("sample.keras")
# model.summary()
# reload_model.summary()

In [8]:
# Prepare the training dataset.
batch_size = 32
(x_train, y_train), (x_test, y_test) = k.datasets.mnist.load_data()
x_train = np.reshape(x_train, (-1, 784)).astype("float32")
x_test = np.reshape(x_test, (-1, 784)).astype("float32")
y_train = k.utils.to_categorical(y_train)
y_test = k.utils.to_categorical(y_test)

# Reserve 10,000 samples for validation.
x_val = x_train[-10000:]
y_val = y_train[-10000:]
x_train = x_train[:-10000]
y_train = y_train[:-10000]

# Prepare the training dataset.
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size).repeat()

# Prepare the validation dataset.
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_dataset = val_dataset.batch(batch_size)

train_dataset_iter = iter(train_dataset)

loss calling in fit() will use (y_true, y_pred, sample_weight) as input, and return a scalar loss value

In [30]:
# the loss class in keras use the API of (y_true, y_pred)
def fitness(y_true, x):
    y_pred = reload_model(x)
    ce_mat = tf.math.multiply(tf.cast(y_pred, tf.float32), tf.cast(y_true, tf.float32))
    return tf.reduce_sum(-tf.math.log(tf.reduce_sum(ce_mat, axis=-1) + 1e-8))

def datasetFetchingScaling(x):
    return(x * 2 - 1)

In [24]:
def tr_dataset_iter():
    tr_imgs, tr_labs = next(train_dataset_iter)
    tr_imgs = datasetFetchingScaling(tr_imgs)
    yield (tr_labs, tr_imgs)
tr_iter = tr_dataset_iter()

tr_test_loader = next(tr_iter)
fitness(tr_test_loader[0], tr_test_loader[1])

hello


<tf.Tensor: shape=(), dtype=float32, numpy=560.81055>

In [38]:
opt = tf.keras.optimizers.AdamW(learning_rate=1e-4)

# 放其使用fit函式，因為該函式無法覆蓋複雜的倒傳遞計算。若要依照GAN範例重寫train_step，可能會破壞原本
# 進行distribute運算的單元。最終工程與重寫一整個customize training loop相當。

# 目標先改回使用customize training step
# Keras3把optimizer的minimize函數移除，只能使用gradient tape

def training_step(input_data):
    labs, imgs = input_data
    def loss():
        return fitness(labs, imgs)
    opt.minimize(loss, reload_model.trainable_variables)

training_step(tr_test_loader)

AttributeError: 'AdamW' object has no attribute 'minimize'